Download necessay dependencies (if running on desktop instead of Google Colab)

In [ ]:
%pip install seaborn
%pip install numpy
%pip install pandas
%pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Import the necessary dependencies

In [113]:
import seaborn as sns
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import shapiro

from sklearn.tree import DecisionTreeRegressor

Load preprocessed dataset

In [114]:
bike_data = pd.read_csv('./data/SeoulBikeData_Processed.csv', encoding='unicode_escape')


## Training Our Second Model (Decision Tree)

Splitting the Data

In [115]:
x_train, x_test, y_train, y_test = train_test_split(bike_data.drop(['Rented Bike Count'], axis=1), bike_data['Rented Bike Count'], test_size=0.2, random_state=21)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(7008, 13)
(1752, 13)
(7008,)
(1752,)


We didn't scale

In [ ]:
regressor = DecisionTreeRegressor(criterion="squared_error", random_state=21)

regressor.fit(x_train, y_train)

yhat_train = regressor.predict(x_train)
yhat_test = regressor.predict(x_test)

train_mse = mean_squared_error(y_train, yhat_train)
test_mse = mean_squared_error(y_test, yhat_test)

# print(y_test)
# print(yhat_test)

y_test_df = pd.DataFrame(y_test)
yhat_test_df = pd.DataFrame(yhat_test)

print(y_test_df.head())
print("===========================================")
print(yhat_test_df.head())

print('Train MSE:', train_mse)
print('Test MSE:', test_mse)

n_nodes = regressor.tree_.node_count 
print(f"Number of nodes: {n_nodes}") 
depth = regressor.get_depth() 
print(f"Depth of the tree: {depth}")
print(" Parameters: ", regressor.get_params())



      Rented Bike Count
6771                395
5878               1375
2905                453
6167               1165
6431               1357
        0
0   504.0
1  1421.0
2   502.0
3  1103.0
4  1155.0
Train MSE: 0.0
Test MSE: 87543.27739726027
Number of nodes: 13347
Depth of the tree: 32
 Parameters:  {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': None, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 21, 'splitter': 'best'}


In [ ]:
for depth in range(1, 20):
    regressor = DecisionTreeRegressor(criterion="squared_error", random_state=21, max_depth=depth)
    regressor.fit(x_train, y_train)

    yhat_train = regressor.predict(x_train)
    yhat_test = regressor.predict(x_test)

    train_mse = mean_squared_error(y_train, yhat_train)
    test_mse = mean_squared_error(y_test, yhat_test)

    y_test_df = pd.DataFrame(y_test)
    yhat_test_df = pd.DataFrame(yhat_test)

    print("Depth ", depth, " Parameters: ", regressor.get_params())
    print('Train MSE:', train_mse)
    print('Test MSE:', test_mse)
    n_nodes = regressor.tree_.node_count 
    print(f"Number of nodes: {n_nodes}") 


Depth  1  Parameters:  {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 1, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 21, 'splitter': 'best'}
Train MSE: 314925.87165549694
Test MSE: 322191.75681537244
Number of nodes: 3
Depth  2  Parameters:  {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 2, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 21, 'splitter': 'best'}
Train MSE: 233048.59217233112
Test MSE: 238601.81306663583
Number of nodes: 7
Depth  3  Parameters:  {'ccp_alpha': 0.0, 'criterion': 'squared_error', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'random_state': 21, 'splitter': 

We observe that the training MSE continues to decrease. However, the test MSE continues to decrease until depth 12. Then the test MSE starts increasing. We believe that depth 12 is the optimal max depth and increasing the max depth futher leads to overfitting

Fine-tuning Model Hyperparameters

The following code is based on https://www.geeksforgeeks.org/how-to-tune-a-decision-tree-in-hyperparameter-tuning/  

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [6, 8, 10, 12, 16, 18, 20, 22],
    'min_samples_split': [2, 10, 18, 16, 24, 30],
    'min_samples_leaf': [1, 2, 4, 6, 8, 10],
    'criterion': ['squared_error']
}

dtree_reg = DecisionTreeRegressor(random_state=42) # Initialize a decision tree regressor
grid_search = GridSearchCV(estimator=dtree_reg, param_grid=param_grid, 
                           cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(x_train, y_train)
best_dtree_reg = grid_search.best_estimator_ # Get the best estimator from the grid search
y_pred_train = best_dtree_reg.predict(x_train)
y_pred_test = best_dtree_reg.predict(x_test)

train_mse = mean_squared_error(y_train, y_pred_train)
test_mse = mean_squared_error(y_test, y_pred_test)

best_params = grid_search.best_params_

print(f"Best parameters: {best_params}")
print(f"Train MSE: {train_mse}")
print(f"Test MSE: {test_mse}")

n_nodes = best_dtree_reg.tree_.node_count  
print(f"Number of nodes: {n_nodes}") 

Fitting 5 folds for each of 288 candidates, totalling 1440 fits
Best parameters: {'criterion': 'squared_error', 'max_depth': 20, 'min_samples_leaf': 6, 'min_samples_split': 24}
Train MSE: 33551.060849269306
Test MSE: 65080.173307821824
Number of nodes: 1033


Since there are many hyperparameters, we used a grid search cross validation method to find the optimal hyperparameter combination based on MSE. We found that the best hyperparameter to be a max depth of 20, min sample leaf of 6 and min sample split of 24. 

Comparing the training and testing MSE, the test MSE is higher than we anticipated. This is likely due to our model not being the best fit for the problem and/or our hyperparameter tuning wasn't exhaustive enough. Our test MSE is almost double than our training MSE which suggests a bad fitting of our model.

Compared to our first model (Polynomial Regression), we believe our second model (Decision Tree), is more complex. Out hyperparameter-tuned model has 1033 nodes where decisions are made, while for polynomial regression a degree of two was achieving a good fit. For our next model, we want to look at neural networks. We hope that the increased complexity of neural networks will allow us to create a model that better predicts the Rented Bike Count feature. We think that neural networks better account for relations between features. In turn, this will aide in our MSE.

### Conclusion

We have found that decision trees are not the optimal model for our research question. Even when we tune our hyperparameter, the MSE for our train and test were still relatively large. The test MSE was much larger than the train MSE which points to our model being a bad fit for our problem. Therefore, our first model, polynomial regression, was much better than our second model, decision tree. 

To improve our decision tree model, we can try to do a more exhaustive hyperparameter tuning search. We can increase the granularity and the number of features tested in our hyperparameter tuning. We currently only tune the max depth, min sample in each leaf node, and min samples for splitting at a decision node since we believe those to be the most influential.